# Prepare MosMedData

## Get data downloaded

In [ ]:
!wget -P /mnt/data/machine-learning http://storage.yandexcloud.net/covid19.1110/prod/COVID19_1110.zip
!unzip /mnt/data/machine-learning/COVID19_1110.zip -d /mnt/LSDF/users/zharov

## Prepare unsupervised dataset

In [1]:
!mkdir /mnt/LSDF/users/zharov/COVID19_1110/studies/prepared

mkdir: cannot create directory ‘/mnt/LSDF/users/zharov/COVID19_1110/studies/prepared’: File exists


In [11]:
import tifffile
from medpy.io import load as medload
import numpy as np
from glob import glob
from skimage.exposure import rescale_intensity
from tqdm.auto import tqdm
import os
import re

inner_medload = lambda a: np.swapaxes(medload(a)[0], 0, -1)

In [9]:
all_addrs = glob('/mnt/LSDF/users/zharov/COVID19_1110/studies/CT-*/*.nii.gz')

In [3]:
extract_experiment_id = re.compile(r'study_(\d+).nii.gz')

In [10]:
for addr in tqdm(all_addrs):
    img = inner_medload(addr)
    filename = os.path.split(addr)[1]
    experiment_id = extract_experiment_id.findall(filename)[0]
    img = rescale_intensity(img, in_range=(-900, 500), out_range=(0, 255)).astype(np.uint8)
    tifffile.imsave(f'/mnt/LSDF/users/zharov/COVID19_1110/studies/prepared/{experiment_id}.tiff', img)

  0%|          | 0/1110 [00:00<?, ?it/s]

## Prepare supervised dataset

In [2]:
extract_mask_id = re.compile(r'study_(\d+)_mask.nii.gz')

In [3]:
!mkdir /mnt/LSDF/users/zharov/COVID19_1110/uniformly_named_masks

mkdir: cannot create directory ‘/mnt/LSDF/users/zharov/COVID19_1110/uniformly_named_masks’: File exists


In [3]:
for addr in tqdm(glob('/mnt/LSDF/users/zharov/COVID19_1110/masks/*.nii.gz')):
    id = extract_mask_id.findall(os.path.split(addr)[1])[0]
    msk = inner_medload(addr).astype(np.uint8)
    tifffile.imsave(f'/mnt/LSDF/users/zharov/COVID19_1110/uniformly_named_masks/{id}.tiff', msk)

  0%|          | 0/50 [00:00<?, ?it/s]

In [4]:
len(glob('/mnt/LSDF/users/zharov/COVID19_1110/uniformly_named_masks/*.tiff'))

50

# Prepare lungs dataset

## Get data downloaded

In [ ]:
!wget -P /mnt/data/machine-learning https://zenodo.org/record/3757476/files/Lung_Mask.zip
!wget -P /mnt/data/machine-learning https://zenodo.org/record/3757476/files/COVID-19-CT-Seg_20cases.zip
    
!unzip /mnt/data/machine-learning/Lung_Mask.zip -d /mnt/LSDF/users/zharov/Lung_Masks
!unzip /mnt/data/machine-learning/COVID-19-CT-Seg_20cases.zip -d /mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases

In [12]:
!mkdir /mnt/LSDF/users/zharov/CTLungs
!mkdir /mnt/LSDF/users/zharov/CTLungs/images
!mkdir /mnt/LSDF/users/zharov/CTLungs/masks

## Prepare images

In [13]:
all_image_addrs = glob('/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/*.nii.gz')

In [34]:
for i, addr in tqdm(enumerate(all_image_addrs), total=len(all_image_addrs)):
    img = inner_medload(addr)
    if img.dtype == np.int16:
        img = rescale_intensity(img, in_range=(-900, 500), out_range=(0, 255)).astype(np.uint8)
    tifffile.imsave(f'/mnt/LSDF/users/zharov/CTLungs/images/{i}.tiff', img)

  0%|          | 0/20 [00:00<?, ?it/s]

/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_009.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_003.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_008.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_006.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_005.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_007.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_002.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_001.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_010.nii.gz conversion
/mnt/LSDF/users/zharov/COVID-19-CT-Seg_20cases/coronacases_004.nii.gz conversion


## Prepare masks

In [32]:
for i, addr in tqdm(enumerate(all_image_addrs), total=len(all_image_addrs)):
    addr = os.path.join('/mnt/LSDF/users/zharov/Lung_Masks', os.path.split(addr)[1])
    img = inner_medload(addr).astype(np.uint8)
    tifffile.imsave(f'/mnt/LSDF/users/zharov/CTLungs/masks/{i}.tiff', img)

  0%|          | 0/20 [00:00<?, ?it/s]